<a href="https://colab.research.google.com/github/atreyid/COVID_19_Tech_Stocks_Analysis/blob/master/Atreyi_Dasmahapatra_p3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 3: Poisonous Mushrooms

In this project, you'll investigate properties of mushrooms. This classic dataset contains over 8000 examples, where each describes a mushroom by a variety of features like color, odor, etc., and the target variable is an indicator for whether the mushroom is poisonous. The feature space has been binarized. Look at the feature_names below to see all 126 binary names.

You'll start by running PCA to reduce the dimensionality from 126 down to 2 so that you can easily visualize the data. In general, PCA is very useful for visualization (though sklearn.manifold.tsne is known to produce better visualizations). Recall that PCA is a linear transformation. The 1st projected dimension is the linear combination of all 126 original features that captures as much of the variance in the data as possible. The 2nd projected dimension is the linear combination of all 126 original features that captures as much of the remaining variance as possible. The idea of dense low dimensional representations is crucial to machine learning!

Once you've projected the data to 2 dimensions, you'll experiment with clustering using k-means and density estimation with Gaussian mixture models (GMM). Finally, you'll train a classifier by fitting a GMM for the positive class and a GMM for the negative class, and perform inference by comparing the probabilities output by each model.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please **prepare your own write-up and write your own code**.

In [ ]:
%matplotlib inline

import urllib.request as urllib2 # For python3
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from matplotlib.colors import LogNorm

In [ ]:
MUSHROOM_DATA = 'https://raw.githubusercontent.com/UCB-MIDS/207-Applied-Machine-Learning/master/Data/mushroom.data'
MUSHROOM_MAP = 'https://raw.githubusercontent.com/UCB-MIDS/207-Applied-Machine-Learning/master/Data/mushroom.map'

Load feature names.

In [ ]:
feature_names = []

for line in urllib2.urlopen(MUSHROOM_MAP):
    [index, name, junk] = line.decode('utf-8').split()
    feature_names.append(name)

print('Loaded feature names: ', len(feature_names))
print(feature_names)

Load data. The dataset is sparse, but there aren't too many features, so we'll use a dense representation, which is supported by all sklearn objects.

In [ ]:
X, Y = [], []

for line in urllib2.urlopen(MUSHROOM_DATA):
    items = line.decode('utf-8').split()
    Y.append(int(items.pop(0)))
    x = np.zeros(len(feature_names))
    for item in items:
        feature = int(str(item).split(':')[0])
        x[feature] = 1
    X.append(x)

# Convert these lists to numpy arrays.
X = np.array(X)
Y = np.array(Y)

# Split into train and test data.
train_data, train_labels = X[:7000], Y[:7000]
test_data, test_labels = X[7000:], Y[7000:]

# Check that the shapes look right.
print(train_data.shape, test_data.shape)

### Part 1:

Do a principal components analysis on the data. Show what fraction of the total variance in the training data is explained by the first k principal components, for k in [1, 2, 3, 4, 5, 10, 20, 30, 40, 50].  Also show a lineplot of fraction of total variance vs. number of principal components, for all possible numbers of principal components.

Notes:
* You can use `PCA` to produce a PCA analysis.

In [ ]:
def P1():
    
    pca = PCA()
    pca.fit_transform(train_data)
    
    '''
    Need to compute percentage of variance explained by each of the selected components
    Using only pca.explained_variance_ratio_[i] gives the variance explained solely 
    by the i+1st dimension. The model.explained_variance_ratio_.cumsum() returns the 
    cumulative variance explained by the first i+1 dimensions.
    '''
    
    variances = pca.explained_variance_ratio_.cumsum()
    k_values = [1, 2, 3, 4, 5, 10, 20, 30, 40, 50]
    variances_for_selected_k_values = []
    
    for index, k in enumerate(k_values):
        print ("k =", k, " Variance =", round(variances[k],3))
        variances_for_selected_k_values.append(variances[k])
    
    plt.plot(k_values,variances_for_selected_k_values, linewidth=2)
    plt.xlabel("k values")
    plt.ylabel("Variance corresponding to each k")
    plt.title("Fraction of total variance explained by first k components")



P1()

### Part 2:

PCA can be very useful for visualizing data. Project the training data down to 2 dimensions and show as a square scatterplot. Show the positive (poisonous) examples in red and the negative (non-poisonous) examples in green. Here's a reference for plotting: http://matplotlib.org/users/pyplot_tutorial.html

Notes:
* You can use `PCA` to produce a PCA analysis.

In [ ]:
def P2():
    pca = PCA(n_components=2)
    results = pca.fit_transform(train_data)
    #print(results)
    
    plt.figure(figsize = (5,5))
    plt.axis([-3,3,-3,3])

    #when train_label == 1, then the mushroom is poisonous
    #when train_label == 0, then the mushroom is not poisonous
    plt.plot(results[:,0][train_labels==1], results[:,1][train_labels==1], 'go', markersize=2)
    plt.plot(results[:,0][train_labels==0], results[:,1][train_labels==0], 'ro', markersize=2)
    
    plt.xlabel('Component 1')
    plt.ylabel('Component 2')
    plt.title("PCA with 2 dimensions visualization")
    plt.legend(["Poisonous", "Non poisonous"])
    
P2()

Part 3:

Fit a k-means cluster model with 6 clusters over the 2d projected data. As in part 2, show as a square scatterplot with the positive (poisonous) examples in red and the negative (non-poisonous) examples in green.  For each cluster, mark the centroid and plot a circle that goes through the cluster's example that is most distant from the centroid.

Notes:
* You can use `KMeans` to produce a k-means cluster analysis.
* You can use `linalg.norm` to determine distance (dissimilarity) between observations.

In [ ]:
def P3():
    #first applying PCA to reduce data to 2D
    pca = PCA(n_components=2)
    results = pca.fit_transform(train_data)
    
    #Plotting poisonous vs non-poisonous
    plt.figure(figsize = (10,10))
    plt.plot(results[:,0][train_labels==1], results[:,1][train_labels==1], 'go', markersize=1)
    plt.plot(results[:,0][train_labels==0], results[:,1][train_labels==0], 'ro', markersize=1)

    plt.xlabel('Component 1',fontsize = 16)
    plt.ylabel('Component 2',fontsize = 16)
    plt.title("PCA components=2, KMeans clusters= 6",fontsize = 16)
    plt.legend(["Poisonous", "Non poisonous"],fontsize = 16) 
    
    '''
    For plotting clusters with cluster diameter and a circle that goes 
    through the cluster's example that is most distant from the centroid I used the approach from
    this answer on stackexchange:
    
    https://datascience.stackexchange.com/questions/32753/
    find-cluster-diameter-and-associated-cluster-points-with-kmeans-clustering-scik
    '''

    kmeans = KMeans(n_clusters=6).fit(results)
    kcenters = kmeans.cluster_centers_
    kpredictions = kmeans.predict(results)
    plt.scatter(kcenters[:, 0], kcenters[:, 1], c='black', s=30, zorder=3)
    ax = plt.gca()

    for idx in range(6):
        radii = max([np.linalg.norm(np.subtract(i,kcenters[idx])) 
                   for i in zip(results[kpredictions == idx, 0],results[kpredictions == idx, 1])])
        ax.add_patch(plt.Circle(kcenters[idx],radii,fill=False,alpha=0.5))


P3()

### Part 4:

Fit Gaussian mixture models for the positive (poisonous) examples in your 2d projected data. Vary the number of mixture components from 1 to 4 and the covariance matrix type 'spherical', 'diag', 'tied', 'full' (that's 16 models).  Show square plots of the estimated density contours presented in a 4x4 grid - one row each for a number of mixture components and one column each for a convariance matrix type.  

Notes:
* You can use `GaussianMixture(n_components=..., covariance_type=..., random_state=12345)` to produce a Gaussian mixture model.
* You can use `contour` in combination with other methods to plot contours, like in this example: http://scikit-learn.org/stable/auto_examples/mixture/plot_gmm_pdf.html#example-mixture-plot-gmm-pdf-py
* You can use `contour` without the `norm` and `levels` parameters. 

In [ ]:
def P4():
    #first applying PCA to reduce data to 2D
    pca = PCA(n_components=2)
    result_positives = pca.fit_transform(train_data)[train_labels == 1]
    cov_matrix_type = ['spherical', 'diag', 'tied', 'full']
    count = 1
    
    fig = plt.figure() #initializing plot
    fig.set_size_inches(25,21)
    fig.suptitle('Part 4: Gaussian Mixture Model (GMM) for 1-4 Mixture Components, Covariance Matrix Types spherical/diag/tied/full, Positive Examples Only', size=16)
    plt.subplots_adjust(left=0.125, bottom= 0.1, right=0.9, top=0.9, wspace=0.1, hspace=0.3)
    
    for component in range(1,5):
        for cov_type in cov_matrix_type:
            gmm  = GaussianMixture(n_components=component, covariance_type=cov_type)
            gmm.fit(result_positives)
            
            gmm_predict = gmm.predict(result_positives)
            x = np.linspace(-2.5, 3.5)
            y = np.linspace(-3., 3.)
            X, Y = np.meshgrid(x, y)
            XX = np.array([X.ravel(), Y.ravel()]).T
            Z = -gmm.score_samples(XX)
            Z = Z.reshape(X.shape)
            
            ax = fig.add_subplot(4, 4, count) #add to 4*4 grid
            ax.set_xlabel('PC1')
            ax.set_ylabel('PC2')
            count = count + 1
        
        
            CS = plt.contour(X, Y, Z, norm=LogNorm(vmin=1.0, vmax=1000.0),
                             levels=np.logspace(0, 2, 10), alpha =0.3, cmap = 'hot')
            CB = plt.colorbar(CS, shrink=0.8)
            CB.ax.tick_params(labelsize=0) #removing labels on tick bar
            plt.scatter(result_positives[:, 0], result_positives[:, 1], c=gmm_predict,marker="4",  cmap='cool')
            plt.title("GMM with " + str(component) + " mixture component\nCovariance matrix type: " + cov_type)
            plt.grid()
            plt.axis('tight')
            
P4()

### Part 5:

Fit two Gaussian mixture models, one for the positive examples and one for the negative examples in your 2d projected data. Use 4 mixture components and full convariance for each model.  Predict the test example labels by picking the labels corresponding to the larger of the two models' probabilities.  What is the accuracy of you predictions on the test data?

Notes:
* You can use `GaussianMixture(n_components=..., covariance_type=..., random_state=12345)` to produce a Gaussian mixture model.
* You can use `GaussianMixture`'s `score_samples` method to find the probabilities.

In [ ]:
def P5():
    pca = PCA(n_components=2)
    
    #Apply dimentionality reduction to train data
    train_results = pca.fit_transform(train_data)
    
    #Apply dimentionality reduction to test data
    test_results = pca.transform(test_data)
    
    train_positives = train_results[train_labels == 1]
    train_negatives = train_results[train_labels == 0]
    
    gmm_positives = GaussianMixture(n_components=4, covariance_type="full").fit(train_positives)
    gmm_negatives = GaussianMixture(n_components=4, covariance_type="full").fit(train_negatives)

    # Predict the test example labels by picking the labels corresponding 
    # to the larger of the two models' probabilities
    pred_labels = gmm_positives.score_samples(test_results) > gmm_negatives.score_samples(test_results)
    
    #Calculating accuracy of predictions
    accuracy=np.sum(pred_labels==test_labels)/len(test_labels)
    
    print("Accuracy of predictions on the test data is", (round(accuracy,4)))
    
P5()

### Part 6:

Run a series of experiments to find the Gaussian mixture model that results in the best accuracy with no more than 50 parameters.  Do this by varying the number of PCA components, the number of GMM components, and the covariance type.

Notes:
* You can use `GaussianMixture(n_components=..., covariance_type=..., random_state=12345)` to produce a Gaussian mixture model.


* For spherical, diag, and full covariance types:
  * number of parameters = (number of parameters per gmm component * number of gmm components - 1) * number of classes
  * number of parameters per gmm component includes all the means plus all the non-zero, non-duplicated values in the covariance matrix plus the mixing weight
  * Each mixing weight parameter indicates how much to weight a particular gmm component; the -1 above accounts for the fact that the mixing weights must sum to 1, so you do not need to include the last mixing weight as its own parameter


* To calculate the number of parameters for tied covariance type:
  * number of parameters = (number of parameters per class - 1) * number of classes
  * number of parameters per class includes all the means and mixing weights for all the gmm components plus all the non-zero, non-duplicated values in the one shared covariance matrix
  * Each mixing weight parameter indicates how much to weight a particular gmm component; the -1 above accounts for the fact that the mixing weights must sum to 1, so you do not need to include the last mixing weight as its own parameter

In [ ]:
def P6():
    
    #I will use the previous function P5 in a loop
    def test_PCA_GMM(n_pca, n_gmm, covariance_type):
        pca = PCA(n_components=n_pca)
    
        #Apply dimentionality reduction to train data
        train_results = pca.fit_transform(train_data)

        #Apply dimentionality reduction to test data
        test_results = pca.transform(test_data)

        train_positives = train_results[train_labels == 1]
        train_negatives = train_results[train_labels == 0]

        gmm_positives = GaussianMixture(n_components=n_gmm, covariance_type=covariance_type).fit(train_positives)
        gmm_negatives = GaussianMixture(n_components=n_gmm, covariance_type=covariance_type).fit(train_negatives)

        # Predict the test example labels by picking the labels corresponding 
        # to the larger of the two models' probabilities
        pred_labels = gmm_positives.score_samples(test_results) > gmm_negatives.score_samples(test_results)

        #Calculating accuracy of predictions
        accuracy=np.sum(pred_labels==test_labels)/len(test_labels)
        
        return accuracy
    
    
#     #covariance matrix types
    cov_matrix_type = ["spherical", "diag", "tied", "full"]
    
    #initialize optimum parameters
    optimum_pca_component = 0
    optimum_gmm_component = 0
    optimum_covariance_matrix = 0
    optimum_accuracy = 0
    
    '''
    Tried range till 25, but since the best combination of n_pca and n_gmm doesnot exceed 10, 
    I changed the range to 10 for faster computation
    '''
    for n_pca in range(1,10):
        for n_gmm in range(1,10):
            for cov_type in cov_matrix_type:
                #calculate total number of parameters
                if cov_type == "full" or cov_type == "tied":
                    total_parameters = (n_pca*n_gmm + n_pca*(n_pca + 1)/2* n_gmm) * 2
                if cov_type == "diagonal":
                    total_parameters = (n_pca*n_gmm + n_pca* n_gmm) * 2
                if cov_type == "spherical":
                    total_parameters = (n_pca*n_gmm + n_gmm) * 2
                
                if total_parameters <= 50:
                    calculate_accuracy = test_PCA_GMM(n_pca, n_gmm,cov_type)
                    if calculate_accuracy > optimum_accuracy:
                        optimum_accuracy = calculate_accuracy
                        optimum_gmm_component = n_gmm
                        optimum_pca_component = n_pca
                        optimum_covariance_matrix = cov_type
                        
    print("The combination of parameters that resulted in the best accuracy is as follows:")
    print("Optimum number of PCA components: " + str(optimum_pca_component))
    print("Optimum number of GMM components: " + str(optimum_gmm_component))
    print("Covariance Type: " + str(optimum_covariance_matrix))
    print("Max accuracy: ", round(optimum_accuracy,4))

P6()